In [1]:
from functools import partial
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.air import session
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import math
import scipy.stats as stats
import ray as ray

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(np.array(X), dtype=torch.float32)  # Convert X and y to PyTorch tensors
        self.y = torch.tensor(np.array(y), dtype=torch.float32).reshape(-1, 3)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = {'X': self.X[idx], 'y': self.y[idx]}
        return sample

In [4]:
def load_data(data_dir="./"):

    data = pd.read_csv('~/ML_model/data_0918_PCA.csv')
    X = data[['L_gp', 'R_gp', 'Thick_gp', 'SFP_Loc', 'SFP_Len', 'Thick_sfp', 'Oxide_Thick']]
    y = data[data.columns[8:11]]
    
    # train-test split for model evaluation
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y,
                                                                train_size=0.8,
                                                                shuffle=True, random_state=0)

    # Standardizing data
    scaler = StandardScaler()
    scaler.fit(X_train_raw)
    X_train = scaler.transform(X_train_raw)
    X_test = scaler.transform(X_test_raw)
    
    trainset = MyDataset(X_train, y_train)
    testset = MyDataset(X_test, y_test)
     
    
    return trainset, testset

In [5]:
# Define the model version1
class NeuralNetwork(nn.Module):
    def __init__(self, l1=100, l2=50, l3=15, dr=0.2):
        super().__init__()
        self.flatten = nn.Flatten()
        self.SharedLinear = nn.Linear(7, l1)
        self.NN_stack_1 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, l2),
            nn.ELU(),
            nn.LayerNorm(l2),
            nn.Dropout(p=dr),
            nn.Linear(l2, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        )
        self.NN_stack_2 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        )
        self.NN_stack_3 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, l2),
            nn.ELU(),
            nn.LayerNorm(l2),
            nn.Dropout(dr),
            nn.Linear(l2, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        ) 

    def forward(self, x):
        x = self.flatten(x)
        x = self.SharedLinear(x)
        #x = self.Tanh1(x)
        GaN_NN_output1 = self.NN_stack_1(x)
        GaN_NN_output2 = self.NN_stack_2(x)
        GaN_NN_output3 = self.NN_stack_3(x)
        return GaN_NN_output1, GaN_NN_output2, GaN_NN_output3

In [19]:
# Define the model version 2
class NeuralNetwork(nn.Module):
    def __init__(self, l1=100, l2=50, l3=15, dr=0.2):
        super().__init__()
        self.flatten = nn.Flatten()
        self.SharedLinear = nn.Linear(7, l1)
        self.NN_stack_1 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, l2),
            nn.ELU(),
            nn.LayerNorm(l2),
            nn.Dropout(p=dr),
            nn.Linear(l2, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        )
        self.NN_stack_2 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, 20),
            nn.Softplus(),
            nn.LayerNorm(20),
            nn.Dropout(p=dr),
            nn.Linear(20, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        )
        self.NN_stack_3 = nn.Sequential(
            nn.Softplus(),
            nn.LayerNorm(l1),
            nn.Dropout(p=dr),
            nn.Linear(l1, l2),
            nn.ELU(),
            nn.LayerNorm(l2),
            nn.Dropout(dr),
            nn.Linear(l2, l3),
            nn.Softplus(),
            nn.LayerNorm(l3),
            nn.Dropout(dr),
            nn.Linear(l3, 1)
        ) 

    def forward(self, x):
        x = self.flatten(x)
        x = self.SharedLinear(x)
        #x = self.Tanh1(x)
        GaN_NN_output1 = self.NN_stack_1(x)
        GaN_NN_output2 = self.NN_stack_2(x)
        GaN_NN_output3 = self.NN_stack_3(x)
        return GaN_NN_output1, GaN_NN_output2, GaN_NN_output3

In [7]:
def train_cifar(config, data_dir=None, max_num_epochs=10):
    df = pd.read_csv('~/ML_model/data_0918_PCA.csv')
    net = NeuralNetwork(config["l1"], config["l2"], config["l3"], config["dr"])
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
    net.to(device)

    criterion = nn.MSELoss().to(device)
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], weight_decay=0.2)

    checkpoint = session.get_checkpoint()

    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        net.load_state_dict(checkpoint_state["net_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0
    start_epoch = 0
    #data_dir = ''
    trainset, testset = load_data()

    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0
    )
    valloader = torch.utils.data.DataLoader(
        testset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0
    )
    
    best_mse = np.inf   # init to infinity
    for epoch in range(start_epoch, max_num_epochs):  # loop over the dataset multiple times
        #net.train().to(device)
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data['X'], data['y']
            inputs, labels = inputs.to(device), labels.to(device)

            # forward + backward + optimize
 
            y_pred_1, y_pred_2, y_pred_3 = net(inputs)
            loss_1 = criterion(y_pred_1, labels[:, 0].reshape(-1, 1))
            loss_2 = df['BV'].mean()/df['Leakage_2'].mean() * criterion(y_pred_2, labels[:, 1].reshape(-1, 1))
            loss_3 = df['BV'].mean()/df['Cgd_2'].mean() * criterion(y_pred_3, labels[:, 2].reshape(-1, 1))
            loss = (loss_1 + loss_2 + loss_3)/3
            running_loss += loss.item()
            
            # zero the parameter gradients
            optimizer.zero_grad()
            #loss = criterion(outputs, labels)
            (loss_1 + loss_2 + loss_3).backward()
            optimizer.step()
            # print statistics
            #loss = running_loss
            epoch_steps += 1
            
            if i % 200 == 199:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0               
      
        # Validation loss
        net.eval()
        val_loss = 0.0
        val_steps = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data['X'], data['y']
                inputs, labels = inputs.to(device), labels.to(device)

                y_pred_1, y_pred_2, y_pred_3 = net(inputs)
                loss_1 = criterion(y_pred_1, labels[:, 0].reshape(-1, 1))
                loss_2 = df['BV'].mean()/df['Leakage_2'].mean() * criterion(y_pred_2, labels[:, 1].reshape(-1, 1))
                loss_3 = df['BV'].mean()/df['Cgd_2'].mean() * criterion(y_pred_3, labels[:, 2].reshape(-1, 1))
                
                loss = (loss_1 + loss_2 + loss_3)/3
                val_loss += loss.cpu().numpy() * len(y_pred_1)
                val_steps += len(y_pred_1)
        if np.sqrt(val_loss / val_steps) < best_mse:
            best_mse = np.sqrt(val_loss / val_steps)
            checkpoint_dir = '/homes/xiang/ML_model/data'
            checkpoint_path = checkpoint_dir + "/model.checkpoint"
            torch.save((net.state_dict(), optimizer.state_dict()), checkpoint_path)
            # [3] Report metrics and checkpoint.
        ray.train.report({"loss": best_mse}, checkpoint=Checkpoint.from_directory(checkpoint_dir))
    #print(val_loss)
    print("Finished Training")

In [8]:
def test_model(net, device='cpu'):
    model = net
    criterion = nn.MSELoss().to(device)
    net.eval()
    data = pd.read_csv('~/ML_model/data_0918_PCA.csv')
    X = data[['L_gp', 'R_gp', 'Thick_gp', 'SFP_Loc', 'SFP_Len', 'Thick_sfp', 'Oxide_Thick']]
    y = data[data.columns[8:11]]
    # train-test split for model evaluation
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y,
                                                                train_size=0.8,
                                                                shuffle=True, random_state=0)

    # Standardizing data
    scaler = StandardScaler()
    scaler.fit(X_train_raw)
    X_train = scaler.transform(X_train_raw)
    X_test = scaler.transform(X_test_raw)
    X_all = scaler.transform(X)

    # Convert to 2D PyTorch tensors
    X_train = torch.tensor(np.array(X_train), dtype=torch.float32).to(device)
    y_train = torch.tensor(np.array(y_train), dtype=torch.float32).reshape(-1, 3)
    X_all = torch.tensor(np.array(X_all), dtype=torch.float32).to(device)

    y_train_1 = y_train[:, 0].reshape(-1, 1).to(device)
    y_train_2 = y_train[:, 1].reshape(-1, 1).to(device)
    y_train_3 = y_train[:, 2].reshape(-1, 1).to(device)

    X_test = torch.tensor(np.array(X_test), dtype=torch.float32).to(device)
    y_test = torch.tensor(np.array(y_test), dtype=torch.float32).reshape(-1, 3)
    y_test_1 = y_test[:, 0].reshape(-1, 1).to(device)
    y_test_2 = y_test[:, 1].reshape(-1, 1).to(device)
    y_test_3 = y_test[:, 2].reshape(-1, 1).to(device)

    y_all = torch.tensor(np.array(y), dtype=torch.float32).reshape(-1, 3)
    y_all_1 = y_all[:, 0].reshape(-1, 1).to(device)
    y_all_2 = y_all[:, 1].reshape(-1, 1).to(device)
    y_all_3 = y_all[:, 2].reshape(-1, 1).to(device)
    
    y_pred_BV, y_pred_leak, y_pred_Cgd = model(X_all)
    y_test_BV, y_test_leak, y_test_Cgd = model(X_test)
    y_train_BV, y_train_leak, y_train_Cgd = model(X_train)
    
    rmse_BV = math.sqrt(np.square(y_test_BV.cpu().detach().numpy().reshape(len(y_test_BV))
                                  - y_test_1.cpu().detach().numpy().reshape(len(y_test_BV))).mean())
    rmse_Leak = math.sqrt(np.square(y_test_leak.cpu().detach().numpy().reshape(len(y_test_BV))
                                  - y_test_2.cpu().detach().numpy().reshape(len(y_test_BV))).mean())
    rmse_Cgd = math.sqrt(np.square(y_test_Cgd.cpu().detach().numpy().reshape(len(y_test_BV))
                                  - y_test_3.cpu().detach().numpy().reshape(len(y_test_BV))).mean())

    print('Testing RMSE: ', rmse_BV, rmse_Leak, rmse_Cgd)
    

    rmse_BV_train = math.sqrt(np.square(y_train_BV.cpu().detach().numpy().reshape(len(y_train_BV))
                              - y_train_1.cpu().detach().numpy().reshape(len(y_train_BV))).mean())
    rmse_Leak_train = math.sqrt(np.square(y_train_leak.cpu().detach().numpy().reshape(len(y_train_BV))
                              - y_train_2.cpu().detach().numpy().reshape(len(y_train_BV))).mean())
    rmse_Cgd_train = math.sqrt(np.square(y_train_Cgd.cpu().detach().numpy().reshape(len(y_train_BV))
                              - y_train_3.cpu().detach().numpy().reshape(len(y_train_BV))).mean())
    
    print('Training RMSE: ', rmse_BV_train, rmse_Leak_train, rmse_Cgd_train)
    
    

    # Calculate the r squared value
    r_squared_1 = stats.pearsonr(y_all_1.cpu().detach().numpy().reshape(len(y_all_1)),
                                 y_pred_BV.cpu().detach().numpy().reshape(len(y_all_1)))[0]
    r_squared_2 = stats.pearsonr(y_all_2.cpu().detach().numpy().reshape(len(y_all_1)),
                                 y_pred_leak.cpu().detach().numpy().reshape(len(y_all_1)))[0]
    r_squared_3 = stats.pearsonr(y_all_3.cpu().detach().numpy().reshape(len(y_all_1)),
                                 y_pred_Cgd.cpu().detach().numpy().reshape(len(y_all_1)))[0]

    print('R_squared: ', r_squared_1, r_squared_2, r_squared_3)
    return

# No Opt Algorithm

In [26]:
[2**i for i in range(5)]

[1, 2, 4, 8, 16]

In [78]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = "~/ML_model/data/"
    #load_data(data_dir)

    config = {
        "l1": tune.choice([256,128,64]),
        "l2": tune.choice([8]),
        "l3": tune.choice([16]),
        "lr": tune.loguniform(1e-6, 1e-3),
        "batch_size": tune.choice([2, 4, 8, 16, 32, 64]),
    }
#     config = {
#         "l1": tune.choice([2**i for i in range(9)]),
#         "l2": tune.choice([2**i for i in range(7)]),
#         "l3": tune.choice([2**i for i in range(5)]),
#         "lr": tune.loguniform(1e-6, 1e-4),
#         "batch_size": tune.choice([4, 8, 16, 32]),
#     }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
    result = tune.run(
        partial(train_cifar, data_dir=data_dir, max_num_epochs=max_num_epochs),
        resources_per_trial={"cpu": 18, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    #print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = NeuralNetwork(best_trial.config["l1"], best_trial.config["l2"], best_trial.config["l3"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
    #print(f"Using {device} device")
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_trial.checkpoint.to_directory(), "model.checkpoint")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)
    torch.save(best_trained_model, 'Ray_GaN_Best_NoOpt.pth')
    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))

if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=25, max_num_epochs=4000, gpus_per_trial=1)

2023-09-21 12:43:44,792	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2023-09-21 12:43:46,614	ERROR tune_controller.py:1502 -- Trial task failed for trial train_cifar_07712_00000
Traceback (most recent call last):
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/worker.py", line 2547, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(KeyError): ray::ImplicitFunc.train() (pid=1826438, ip=137.203.141.211, actor_id

Trial name
train_cifar_07712_00000
train_cifar_07712_00001
train_cifar_07712_00002
train_cifar_07712_00003
train_cifar_07712_00004
train_cifar_07712_00005


2023-09-21 12:43:48,553	ERROR tune_controller.py:1502 -- Trial task failed for trial train_cifar_07712_00001
Traceback (most recent call last):
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/worker.py", line 2547, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(KeyError): ray::ImplicitFunc.train() (pid=1826487, ip=137.203.141.211, actor_id

2023-09-21 12:43:56,573	ERROR tune_controller.py:1502 -- Trial task failed for trial train_cifar_07712_00005
Traceback (most recent call last):
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/ray/_private/worker.py", line 2547, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(KeyError): ray::ImplicitFunc.train() (pid=1826679, ip=137.203.141.211, actor_id

2023-09-21 12:43:58,409	WARNING experiment_analysis.py:596 -- Could not find best trial. Did you pass the correct `metric` parameter?


AttributeError: 'NoneType' object has no attribute 'config'

In [35]:
model = torch.load('Ray_GaN_Best_NoOpt.pth')

In [36]:
test_model(model, 'cuda:0')

Testing RMSE:  63.29068216743085 1.3491744625595956 2.134257850392133
Training RMSE:  54.97884003471245 1.1240362701876607 2.111313444892882
R_squared:  0.4033425149732184 0.6197252962314967 0.9874024557123837


In [1]:
best_trial = result.get_best_trial("loss", "min", "last")

NameError: name 'result' is not defined

# Optuna Algorithm Weight decay 0.01

In [9]:
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter

#algo = ConcurrencyLimiter(algo, max_concurrent=4)

In [20]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = "~/ML_model/data/"
    load_data(data_dir)
    algo = OptunaSearch()
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
#     config = {
#         "l1": tune.choice([2**i for i in range(9)]),
#         "l2": tune.choice([2**i for i in range(7)]),
#         "l3": tune.choice([2**i for i in range(5)]),
#         "lr": tune.loguniform(1e-6, 1e-4),
#         "batch_size": tune.choice([4, 8, 16, 32]),
#     }
    config = {
        "l1": tune.choice([200]),
        "l2": tune.choice([100]),
        "l3": tune.choice([15]),
        "lr": tune.loguniform(1e-6, 1e-3),
        "dr": tune.choice([0.4]),
        "batch_size": tune.choice([8]),
    }
    scheduler = ASHAScheduler(
        #metric="loss",
        #mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
#     result = tune.run(
#         partial(train_cifar, data_dir=data_dir),
#         #metric="loss",
#         #mode="min",
#         resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
#         #search_alg=algo,
#         #config=config,
#         num_samples=num_samples,
#         scheduler=scheduler,
#     )
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar, data_dir=data_dir, max_num_epochs=max_num_epochs),
            resources={"cpu": 18, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            search_alg=algo,
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    result = tuner.fit()

    best_trial = result.get_best_result("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.metrics['loss']}")
    #print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = NeuralNetwork(best_trial.config["l1"],
                                       best_trial.config["l2"],
                                       best_trial.config["l3"],
                                       best_trial.config["dr"])
    device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
    )
    #print(f"Using {device} device")
    best_trained_model.to(device)

    #best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
    #best_checkpoint_data = best_checkpoint.to_dict()
    checkpoint_path = os.path.join(best_trial.checkpoint.to_directory(), "model.checkpoint")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)
    torch.save(best_trained_model, 'Ray_GaN_Best_Opt.pth')
    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=50, max_num_epochs=4000, gpus_per_trial=1)

(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if is_sparse(pd_dtype):
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if is_sparse(pd_dtype) or not is_extension_array_dtype(pd

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000019)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000020)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000021)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.40

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000050)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000051)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000052)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.40

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000081)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000082)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000083)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.40

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000112)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000113)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-43-43/checkpoint_000114)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_3a6c11ab_1_batch_size=8,dr=0.40

...

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_42287dfa_42_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0000_2023-10-16_20-49-05/checkpoint_000000)
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if is_sparse(pd_dtype):
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/pytho

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_028902c8_45_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-49-07/checkpoint_000000)
(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_028902c8_45_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0001_2023-10-16_20-49-07/checkpoint_000001)
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/val

(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_c1e64992_48_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0006_2023-10-16_20-49-07/checkpoint_000001)
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=2174965)   if is_sparse(pd_dtype):
(train_cifar pid=2174965) /homes/xiang/miniconda3/envs/torch/lib/pytho

Best trial config: {'l1': 200, 'l2': 100, 'l3': 15, 'lr': 6.591118151258976e-05, 'dr': 0.4, 'batch_size': 8}
Best trial final validation loss: 16.372031461513217


(train_cifar pid=2174965) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-10-16_20-43-43/train_cifar_cd750921_50_batch_size=8,dr=0.4000,l1=200,l2=100,l3=15,lr=0.0003_2023-10-16_20-49-07/checkpoint_000003)


In [20]:
model = torch.load('Ray_GaN_Best_Opt_200_45_15_dr4_ba8_3hid4leak.pth')
test_model(model, 'cuda:0')
!rm -r ~/ray_results

Testing RMSE:  23.178826450134803 0.42342966220259987 1.9945640002444562
Training RMSE:  17.522644215492544 0.2905089941561129 1.602387825049562
R_squared:  0.9574222830987479 0.9772650694245054 0.9924157831560831
rm: cannot remove '/homes/xiang/ray_results': No such file or directory


In [21]:
model = torch.load('Ray_GaN_Best_Opt.pth')
test_model(model, 'cuda:0')
!rm -r ~/ray_results

Testing RMSE:  27.750732412209636 0.5898322552704083 2.1027509881074984
Training RMSE:  20.826250179357856 0.36485301216343763 1.7608954626852131
R_squared:  0.9366688073706368 0.9607558943369523 0.9922836500424504


In [17]:
model = torch.load('Ray_GaN_Best_Opt_200_45_15_dr4_ba8.pth')
test_model(model, 'cuda:0')
!rm -r ~/ray_results

Testing RMSE:  26.25557395248138 0.4996919726230642 1.742841178845498
Training RMSE:  12.705843588826006 0.2918066755944392 1.4163448398278602
R_squared:  0.9675323596733305 0.9726782022498277 0.9952297547866584
rm: cannot remove '/homes/xiang/ray_results': No such file or directory


Ray_GaN_Best_Opt_200_45_15_dr6_ba16.pth

Testing RMSE:  26.143923973692004 0.6253982228967825 2.155444893284976

Training RMSE:  19.409331981517294 0.3733639393264249 1.382655420379614

R_squared:  0.9435742174814608 0.9594011428548699 0.9936484750300789

Ray_GaN_Best_Opt_150_45_15_dr6_ba16.pth

Testing RMSE:  24.842214476917864 0.5157956216746119 2.935866855585516

Training RMSE:  16.879906513262576 0.29173749960504586 2.908359049452916

R_squared:  0.955190770397006 0.9740575413825696 0.9859986433655552

Ray_GaN_Best_Opt_100_30_15_dr8_ba8.pth

Testing RMSE:  24.891821612793326 0.4873585489661165 1.752834272842806

Training RMSE:  13.291348986680855 0.2776409088473372 1.3886504138087599

R_squared:  0.9653346047924056 0.9753635769723445 0.9939817029855158

In [133]:
model.state_dict

<bound method Module.state_dict of NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (SharedLinear): Linear(in_features=7, out_features=150, bias=True)
  (NN_stack_1): Sequential(
    (0): Softplus(beta=1, threshold=20)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=150, out_features=45, bias=True)
    (3): ELU(alpha=1.0)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=45, out_features=15, bias=True)
    (6): Softplus(beta=1, threshold=20)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=15, out_features=1, bias=True)
  )
  (NN_stack_2): Sequential(
    (0): Softplus(beta=1, threshold=20)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=150, out_features=15, bias=True)
    (3): Softplus(beta=1, threshold=20)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=15, out_features=1, bias=True)
  )
  (NN_stack_3): Sequential(
    (0): Softplus(beta=1, threshold=20)
    (1): Dropout(p=0.5, 

# points_to_evaluate weight decay 0.001

In [72]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = "~/ML_model/data/"
    load_data(data_dir)
    #initial_params = [
    #    {"l1": 200, "l2": 45, "l3": 15, "lr": 0.000940641, 'batch_size': 4}
    #]
    algo = OptunaSearch()
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
#     config = {
#         "l1": tune.choice([2**i for i in range(9)]),
#         "l2": tune.choice([2**i for i in range(7)]),
#         "l3": tune.choice([2**i for i in range(5)]),
#         "lr": tune.loguniform(1e-6, 1e-3),
#         "batch_size": tune.choice([2, 4, 8, 16]),
#     }
    config = {
        "l1": tune.choice([200, 150, 100]),
        "l2": tune.choice([45, 30, 20]),
        "l3": tune.choice([15]),
        "lr": tune.loguniform(1e-6, 1e-3),
        "batch_size": tune.choice([4, 8, 16, 32, 64]),
    }
    scheduler = ASHAScheduler(
        #metric="loss",
        #mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
#     result = tune.run(
#         partial(train_cifar, data_dir=data_dir),
#         #metric="loss",
#         #mode="min",
#         resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
#         #search_alg=algo,
#         #config=config,
#         num_samples=num_samples,
#         scheduler=scheduler,
#     )
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar, data_dir=data_dir, max_num_epochs=max_num_epochs),
            resources={"cpu": 18, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            search_alg=algo,
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    result = tuner.fit()

    best_trial = result.get_best_result("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.metrics['loss']}")
    #print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = NeuralNetwork(best_trial.config["l1"],
                                       best_trial.config["l2"],
                                       best_trial.config["l3"])
    device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
    )
    #print(f"Using {device} device")
    best_trained_model.to(device)

    #best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
    #best_checkpoint_data = best_checkpoint.to_dict()
    checkpoint_path = os.path.join(best_trial.checkpoint.to_directory(), "model.checkpoint")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)
    torch.save(best_trained_model, 'Ray_GaN_Best_Opt.pth')
    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=20, max_num_epochs=4000, gpus_per_trial=1)

(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if is_sparse(pd_dtype):
(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if is_sparse(pd_dtype) or not is_extension_array_dtype(pd

(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000027)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000028)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000029)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09

(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000063)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000064)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000065)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09

(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000099)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000100)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000101)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09

(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000130)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000131)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09-21_11-37-44/checkpoint_000132)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_03dfe85d_1_batch_size=32,l1=100,l2=20,l3=15,lr=0.0001_2023-09

...

(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f9de1d2_17_batch_size=4,l1=150,l2=45,l3=15,lr=0.0009_2023-09-21_12-15-12/checkpoint_000062)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f9de1d2_17_batch_size=4,l1=150,l2=45,l3=15,lr=0.0009_2023-09-21_12-15-12/checkpoint_000063)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f9de1d2_17_batch_size=4,l1=150,l2=45,l3=15,lr=0.0009_2023-09-21_12-15-12/checkpoint_000064)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f9de1d2_17_batch_size=4,l1=150,l2=45,l3=15,lr=0.0009_2023-09

(train_cifar pid=1825407) Finished Training


(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f8e72b9_18_batch_size=4,l1=150,l2=45,l3=15,lr=0.0002_2023-09-21_12-24-00/checkpoint_000001)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f8e72b9_18_batch_size=4,l1=150,l2=45,l3=15,lr=0.0002_2023-09-21_12-24-00/checkpoint_000002)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f8e72b9_18_batch_size=4,l1=150,l2=45,l3=15,lr=0.0002_2023-09-21_12-24-00/checkpoint_000003)
(train_cifar pid=1825407) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/homes/xiang/ray_results/train_cifar_2023-09-21_11-37-44/train_cifar_3f8e72b9_18_batch_size=4,l1=150,l2=45,l3=15,lr=0.0002_2023-09

Best trial config: {'l1': 150, 'l2': 45, 'l3': 15, 'lr': 0.0009347396563722516, 'batch_size': 4}
Best trial final validation loss: 15.026224379262825


(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if is_sparse(pd_dtype):
(train_cifar pid=1825407) /homes/xiang/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(train_cifar pid=1825407)   if is_sparse(pd_dtype) or not is_extension_array_dtype(pd

In [73]:
model = torch.load('Ray_GaN_Best_Opt.pth')
test_model(model, 'cuda:0')

Testing RMSE:  25.41020759266527 0.5018740936879185 2.4686715801676695
Training RMSE:  10.560881902686916 0.24959997866858566 1.0388155191922837
R_squared:  0.9716330879333199 0.9769721816608539 0.9945187214553621
